# Transfer Learning with VGG16

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maheshghanta/Codes/blob/master/PyTorch_Tutorials/Tutorial1_Exercises/5.Transfer_Learning_Exercise.ipynb)

This tutorial demonstrates **Transfer Learning** using pre-trained VGG16 for CIFAR-10 classification:
- Using VGG16 pre-trained on ImageNet as feature extractor
- Freezing convolutional layers
- Training only the classifier head
- Complete pipeline with TensorBoard logging
- Performance comparison with models from scratch

In [ ]:
%pip install torch torchvision
%pip install ipywidgets
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install scipy
%pip install -i https://test.pypi.org/simple/ exercise-validation==0.1.7

## Overview: Transfer Learning

**What is Transfer Learning?**
- Use knowledge from one task to solve another
- Leverage pre-trained models on large datasets (ImageNet)
- Fine-tune for your specific task

**Why Transfer Learning?**
- ✅ **Faster training**: Pre-trained features
- ✅ **Better accuracy**: Especially with limited data
- ✅ **Less data needed**: Features already learned
- ✅ **Proven architecture**: Battle-tested on ImageNet

**VGG16 Architecture:**
- 13 convolutional layers
- 3 fully connected layers
- Pre-trained on 1.2M ImageNet images (1000 classes)
- We'll use it as a **feature extractor**

**Our Approach:**
1. Load pre-trained VGG16
2. **Freeze** all convolutional layers (no training)
3. Replace classifier for 10 classes (CIFAR-10)
4. Train **only** the new classifier

## Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from copy import deepcopy

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
from exercise_validation import (
    validate_transfer_ex1,
    validate_transfer_ex2,create_check_button,create_feedback
)

## 🎮 Exercise Validation Framework


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

exercise_scores = {}

def create_feedback(success, message, hint=""):
    color, icon, header = ("#4CAF50", "✓", "Correct!") if success else ("#f44336", "✗", "Not quite right")
    display(HTML(f"""<div style='padding: 15px; border-left: 5px solid {color}; background-color: {color}15; margin: 10px 0;'>
        <h3 style='margin: 0; color: {color};'>{icon} {header}</h3><p style='margin: 10px 0 0 0;'>{message}</p>
        {f"<p style='margin: 10px 0 0 0; font-style: italic;'>💡 Hint: {hint}</p>" if hint else ""}</div>"""))

def create_check_button(exercise_name, validation_func):
    output = widgets.Output()
    button = widgets.Button(description='✓ Check Answer', button_style='primary', icon='check')
    def on_click(b):
        with output:
            clear_output()
            success, message, hint = validation_func()
            create_feedback(success, message, hint)
            if success:
                exercise_scores[exercise_name] = True
                print(f"\n🎯 Progress: {sum(exercise_scores.values())}/{len(exercise_scores)} completed!")
    button.on_click(on_click)
    display(button, output)

print("✓ Validation framework loaded!")


## 1. Data Preparation

**Important**: VGG16 expects images normalized with ImageNet mean/std!

In [ ]:
# VGG16 expects 224x224 images and specific normalization
train_transform = transforms.Compose([
    transforms.Resize(224),  # VGG16 expects 224x224
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10
train_dataset_full = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=test_transform
)

# Split: 80% train, 20% validation
train_size = int(0.8 * len(train_dataset_full))
val_size = len(train_dataset_full) - train_size

train_dataset, val_dataset = random_split(
    train_dataset_full,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {test_dataset.classes}")

In [ ]:
# Create DataLoaders
batch_size = 64  # Smaller batch for larger images

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# Check data shape
sample_image, sample_label = next(iter(train_loader))
print(f"Batch shape: {sample_image.shape}")  # Should be (batch_size, 3, 224, 224)
print(f"Label shape: {sample_label.shape}")

## 2. Load Pre-trained VGG16 and Modify

We'll:
1. Load VGG16 pre-trained on ImageNet
2. Freeze all convolutional layers
3. Replace the classifier for 10 classes

In [ ]:
# Load pre-trained VGG16
print("Loading pre-trained VGG16...")
vgg16 = models.vgg16(pretrained=True)
print("✓ VGG16 loaded successfully!\n")

# Show original architecture
print("Original VGG16 Architecture:")
print(vgg16)

# Count original parameters
total_params = sum(p.numel() for p in vgg16.parameters())
print(f"\nTotal parameters in VGG16: {total_params:,}")

In [ ]:
# Freeze all convolutional layers (features)
print("Freezing convolutional layers...")
for param in vgg16.features.parameters():
    param.requires_grad = False

print("✓ All convolutional layers frozen!")

# Check which layers are frozen
frozen_params = sum(p.numel() for p in vgg16.features.parameters())
print(f"Frozen parameters: {frozen_params:,}")

---
## 🎯 Exercise 1: Freeze Pretrained Layers

**Task:** Load a pretrained ResNet18 and freeze all layers.

**Instructions:**
1. Load pretrained ResNet18
2. Freeze all parameters by setting `requires_grad=False`
3. Count frozen parameters


In [ ]:
# TODO: Load pretrained ResNet18
my_resnet = None  # models.resnet18(pretrained=True)

# TODO: Freeze all parameters
# for param in my_resnet.parameters():
#     param.requires_grad = False

if my_resnet:
    frozen = sum(p.numel() for p in my_resnet.parameters() if not p.requires_grad)
    total = sum(p.numel() for p in my_resnet.parameters())
    print(f"Frozen: {frozen:,} / {total:,}")

create_check_button("transfer_ex1", lambda: validate_transfer_ex1(my_resnet))


---
## 🎯 Exercise 2: Replace Classifier

**Task:** Replace the final fully connected layer for 10 classes.

**Instructions:**
1. Get input features from `my_resnet.fc.in_features`
2. Replace `my_resnet.fc` with new Linear layer for 10 classes


In [ ]:
# Provided: fresh ResNet18 with frozen layers
my_resnet2 = models.resnet18(pretrained=True)
for param in my_resnet2.parameters():
    param.requires_grad = False

# TODO: Replace the classifier
# Get the number of input features
num_features = None  # my_resnet2.fc.in_features

# Replace fc layer for 10 classes
# my_resnet2.fc = nn.Linear(num_features, 10)

if num_features:
    print(f"Input features: {num_features}")
    print(f"Output classes: {my_resnet2.fc.out_features}")

create_check_button("transfer_ex2", lambda: validate_transfer_ex2(my_resnet2, num_features))


In [ ]:
# Replace the classifier for CIFAR-10 (10 classes)
# Original VGG16 classifier is for ImageNet (1000 classes)

print("\nOriginal classifier:")
print(vgg16.classifier)

# Create new classifier
num_features = vgg16.classifier[0].in_features  # 25088

vgg16.classifier = nn.Sequential(
    nn.Linear(num_features, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(4096, 1024),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(1024, 10)  # 10 classes for CIFAR-10
)

print("\n✓ New classifier for CIFAR-10:")
print(vgg16.classifier)

# Count trainable parameters
trainable_params = sum(p.numel() for p in vgg16.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in vgg16.parameters())

print(f"\nParameter Summary:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Frozen parameters: {total_params - trainable_params:,}")
print(f"  Percentage trainable: {100 * trainable_params / total_params:.2f}%")

## 3. Setup Training Components

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model to device
model = vgg16.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer - only for trainable parameters!
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),  # Only trainable params
    lr=0.001,
    weight_decay=1e-4
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Initial learning rate: {optimizer.param_groups[0]['lr']}")
print(f"Scheduler: ReduceLROnPlateau")

## 4. Setup TensorBoard Logging

In [ ]:
# Create TensorBoard writer
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f'runs/vgg16_transfer_{timestamp}'
writer = SummaryWriter(log_dir)

print(f"TensorBoard logs saved to: {log_dir}")
print(f"To view: tensorboard --logdir=runs")

# Log model architecture
sample_input = torch.randn(1, 3, 224, 224).to(device)
writer.add_graph(model, sample_input)
print("Model graph added to TensorBoard")

## 5. Training Functions

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device, epoch, writer):
    """
    Train the model for one epoch
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Log batch metrics
        if batch_idx % 50 == 0:
            writer.add_scalar('Train/BatchLoss', loss.item(), epoch * len(loader) + batch_idx)
            batch_acc = 100. * correct / total
            writer.add_scalar('Train/BatchAccuracy', batch_acc, epoch * len(loader) + batch_idx)
            
            if batch_idx % 100 == 0:
                print(f'  Batch [{batch_idx}/{len(loader)}] | '
                      f'Loss: {loss.item():.4f} | Acc: {batch_acc:.2f}%')
    
    avg_loss = running_loss / len(loader)
    accuracy = 100. * correct / total
    
    return avg_loss, accuracy


@torch.no_grad()
def validate(model, loader, criterion, device, epoch, writer, phase='Validation'):
    """
    Validate the model
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    avg_loss = running_loss / len(loader)
    accuracy = 100. * correct / total
    
    # Log to TensorBoard
    writer.add_scalar(f'{phase}/Loss', avg_loss, epoch)
    writer.add_scalar(f'{phase}/Accuracy', accuracy, epoch)
    
    return avg_loss, accuracy, all_preds, all_labels

print("Training functions defined successfully!")

## 6. Training Loop

Transfer learning typically converges much faster!

In [ ]:
# Training configuration
num_epochs = 15  # Fewer epochs needed for transfer learning
best_val_acc = 0.0

# Store metrics
train_losses = []
train_accs = []
val_losses = []
val_accs = []
learning_rates = []

print(f"Starting transfer learning for {num_epochs} epochs...")
print("Note: Transfer learning converges faster than training from scratch!")
print("=" * 80)

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 80)
    
    # Train
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device, epoch, writer
    )
    
    # Validate
    val_loss, val_acc, val_preds, val_labels = validate(
        model, val_loader, criterion, device, epoch, writer, 'Validation'
    )
    
    # Update learning rate
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Train/LearningRate', current_lr, epoch)
    
    # Store metrics
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    learning_rates.append(current_lr)
    
    # Print summary
    print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(f"Learning Rate: {current_lr:.6f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
        }, 'best_vgg16_transfer.pth')
        print(f"✓ Best model saved! (Val Acc: {val_acc:.2f}%)")
    
    # Early stopping
    if current_lr < 1e-7:
        print("\nLearning rate too small. Early stopping...")
        break

print("\n" + "=" * 80)
print(f"Training completed! Best validation accuracy: {best_val_acc:.2f}%")
writer.close()

## 7. Plot Training Metrics

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_range = range(1, len(train_losses)+1)

# Loss plot
axes[0].plot(epochs_range, train_losses, 'b-', label='Train Loss', marker='o', markersize=5)
axes[0].plot(epochs_range, val_losses, 'r-', label='Val Loss', marker='s', markersize=5)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Transfer Learning: Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(epochs_range, train_accs, 'b-', label='Train Acc', marker='o', markersize=5)
axes[1].plot(epochs_range, val_accs, 'r-', label='Val Acc', marker='s', markersize=5)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Transfer Learning: Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=85, color='g', linestyle='--', alpha=0.5, label='Target: 85%')

# Learning rate
axes[2].plot(epochs_range, learning_rates, 'g-', marker='d', markersize=5)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('Learning Rate', fontsize=12)
axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.suptitle('VGG16 Transfer Learning on CIFAR-10', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('vgg16_transfer_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Training curves saved to 'vgg16_transfer_curves.png'")

## 8. Test Set Evaluation

In [ ]:
# Load best model
checkpoint = torch.load('best_vgg16_transfer.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best model from epoch {checkpoint['epoch']+1}")
print(f"Best validation accuracy: {checkpoint['val_acc']:.2f}%")

# Evaluate on test set
test_loss, test_acc, test_preds, test_labels = validate(
    model, test_loader, criterion, device, 0, writer, 'Test'
)

print("\n" + "=" * 80)
print("TEST SET RESULTS")
print("=" * 80)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")
print("=" * 80)

## 9. Performance Comparison

Let's compare VGG16 transfer learning with our previous models!

In [ ]:
# Performance comparison
models_comparison = {
    'Model': ['MLP\n(Tutorial 3)', 'Custom CNN\n(Tutorial 4)', 'VGG16 Transfer\n(This Tutorial)'],
    'Accuracy': [52, 78, test_acc],  # Approximate values
    'Params': ['1.7M', '1.5M', f'{trainable_params/1e6:.1f}M trainable\n({total_params/1e6:.1f}M total)'],
    'Training Time': ['20 epochs', '30 epochs', f'{len(train_losses)} epochs'],
    'Advantages': [
        'Simple,\nFast training',
        'Better than MLP,\nFewer params',
        'Best accuracy,\nPre-trained features'
    ]
}

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Accuracy comparison
accuracies = [52, 78, test_acc]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = ax1.bar(range(3), accuracies, color=colors, edgecolor='black', linewidth=2)
ax1.set_xlabel('Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('Test Accuracy (%)', fontsize=12, fontweight='bold')
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(range(3))
ax1.set_xticklabels(['MLP', 'Custom CNN', 'VGG16\nTransfer'], fontsize=10)
ax1.set_ylim([0, 100])
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=85, color='g', linestyle='--', linewidth=2, label='Excellent (85%)')
ax1.legend()

# Add value labels
for i, (bar, acc) in enumerate(zip(bars, accuracies)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 2,
             f'{acc:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Add improvement percentage
    if i > 0:
        improvement = acc - accuracies[0]
        ax1.text(bar.get_x() + bar.get_width()/2., height/2,
                 f'+{improvement:.1f}%', ha='center', va='center', 
                 fontsize=10, color='white', fontweight='bold')

# Training efficiency comparison
epochs_list = [20, 30, len(train_losses)]
bars2 = ax2.bar(range(3), epochs_list, color=colors, edgecolor='black', linewidth=2)
ax2.set_xlabel('Model', fontsize=12, fontweight='bold')
ax2.set_ylabel('Epochs to Converge', fontsize=12, fontweight='bold')
ax2.set_title('Training Efficiency', fontsize=14, fontweight='bold')
ax2.set_xticks(range(3))
ax2.set_xticklabels(['MLP', 'Custom CNN', 'VGG16\nTransfer'], fontsize=10)
ax2.grid(axis='y', alpha=0.3)

for bar, epochs in zip(bars2, epochs_list):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{epochs} epochs', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 PERFORMANCE SUMMARY:")
print("=" * 80)
print(f"{'Model':<20} {'Accuracy':<15} {'Improvement':<15} {'Epochs'}")
print("-" * 80)
print(f"{'MLP':<20} {52:>6.1f}%        {'Baseline':<15} {20}")
print(f"{'Custom CNN':<20} {78:>6.1f}%        {'+26.0%':<15} {30}")
print(f"{'VGG16 Transfer':<20} {test_acc:>6.1f}%        {f'+{test_acc-52:.1f}%':<15} {len(train_losses)}")
print("=" * 80)
print(f"\n🏆 Winner: VGG16 Transfer Learning!")
print(f"   - Highest accuracy: {test_acc:.2f}%")
print(f"   - Fastest convergence: {len(train_losses)} epochs")
print(f"   - Leverages pre-trained features from ImageNet")

## 10. Per-Class Accuracy

In [ ]:
# Calculate per-class accuracy
from collections import defaultdict

class_correct = defaultdict(int)
class_total = defaultdict(int)

for pred, label in zip(test_preds, test_labels):
    if pred == label:
        class_correct[label] += 1
    class_total[label] += 1

classes = test_dataset.classes

print("\nPer-Class Accuracy:")
print("-" * 50)
class_accs = []
for i, class_name in enumerate(classes):
    acc = 100.0 * class_correct[i] / class_total[i]
    class_accs.append(acc)
    print(f"{class_name:12s}: {acc:6.2f}% ({class_correct[i]}/{class_total[i]})")

# Plot per-class accuracy
plt.figure(figsize=(14, 6))
bars = plt.bar(range(len(classes)), class_accs, color='mediumseagreen', edgecolor='darkgreen', linewidth=1.5)
plt.xlabel('Class', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Per-Class Accuracy - VGG16 Transfer Learning', fontsize=14, fontweight='bold')
plt.xticks(range(len(classes)), classes, rotation=45, ha='right')
plt.ylim([0, 100])
plt.axhline(y=test_acc, color='r', linestyle='--', linewidth=2, label=f'Overall: {test_acc:.2f}%')
plt.grid(axis='y', alpha=0.3)
plt.legend(fontsize=10)

# Add value labels
for bar, acc in zip(bars, class_accs):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{acc:.1f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('vgg16_per_class_accuracy.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nPer-class accuracy plot saved!")

## 11. Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(test_labels, test_preds)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=classes, yticklabels=classes,
            cbar_kws={'label': 'Count'}, linewidths=0.5)
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - VGG16 Transfer Learning', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('vgg16_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print("=" * 80)
print(classification_report(test_labels, test_preds, target_names=classes, digits=4))

## 12. Sample Predictions with Confidence

In [ ]:
# Get a batch of test images
model.eval()
test_images, test_labels_batch = next(iter(test_loader))
test_images_device = test_images.to(device)

with torch.no_grad():
    outputs = model(test_images_device)
    probabilities = torch.softmax(outputs, dim=1)
    confidences, predictions = probabilities.max(1)

# Move to CPU
test_images = test_images.cpu()
predictions = predictions.cpu()
confidences = confidences.cpu()

# Visualize
fig, axes = plt.subplots(4, 4, figsize=(14, 14))

for i, ax in enumerate(axes.flat):
    # Denormalize
    img = test_images[i].permute(1, 2, 0).numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    
    ax.imshow(img)
    
    # Labels
    true_label = classes[test_labels_batch[i]]
    pred_label = classes[predictions[i]]
    confidence = confidences[i].item() * 100
    
    is_correct = test_labels_batch[i] == predictions[i]
    color = 'green' if is_correct else 'red'
    
    ax.set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%', 
                 color=color, fontsize=9, fontweight='bold')
    ax.axis('off')

plt.suptitle('VGG16 Transfer Learning Predictions\n(Green=Correct, Red=Wrong)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('vgg16_sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("Sample predictions saved!")

## Summary

### Final Results:

| Metric | Value |
|--------|-------|
| **Test Accuracy** | ~85-90% |
| **Training Epochs** | 15 (vs 30 for custom CNN) |
| **Trainable Parameters** | ~21M (vs 138M total) |
| **Improvement over MLP** | +35-38% |
| **Improvement over Custom CNN** | +7-12% |

### Why Transfer Learning Works:

1. **Pre-trained Features**: VGG16 learned general image features on ImageNet
2. **Feature Reusability**: Low-level features (edges, textures) transfer well
3. **Less Data Needed**: Don't need millions of images
4. **Faster Training**: Only train classifier, not entire network
5. **Better Generalization**: Pre-trained features are robust

### Transfer Learning Strategies:

**1. Feature Extraction (What we did)**
- Freeze all pre-trained layers
- Train only new classifier
- Best when: Limited data, similar task

**2. Fine-tuning (Advanced)**
- Unfreeze some layers
- Train with very low learning rate
- Best when: More data available

**3. Full Training**
- Use pre-trained weights as initialization
- Train entire network
- Best when: Large dataset, different task

### When to Use Transfer Learning:

✅ **Use transfer learning when:**
- Limited training data
- Similar domain (images → images)
- Want faster training
- Want better accuracy
- Limited computational resources

❌ **Don't use transfer learning when:**
- Very different domain (text → images)
- Extremely large custom dataset
- Very specific features needed
- Computational resources unlimited

### Popular Pre-trained Models:

- **VGG16/VGG19**: Simple, deep, good baseline
- **ResNet50/101**: Deeper, better accuracy
- **EfficientNet**: Best accuracy/efficiency trade-off
- **MobileNet**: Lightweight, for mobile devices
- **Vision Transformer**: State-of-the-art


### TensorBoard Commands:

```bash
# View all runs together
tensorboard --logdir=runs

# Compare: MLP vs CNN vs Transfer Learning
tensorboard --logdir=runs --port=6006
```